In [1]:
import os
import optuna
from dotenv import load_dotenv
load_dotenv()

/home/jsimonrichard/QML_Research/QFE-Experiments/qfe_exp_venv/lib/python3.10/site-packages/tqdm/auto.py:21: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm


True

In [2]:
optuna.get_all_study_names(storage=os.getenv('OPTUNA_DB'))

['dataset-PROTEINS/embedder-QFE-exp/fold-0',
 'dataset-PROTEINS/embedder-QFE-probs/fold-0',
 'dataset-PROTEINS/embedder-MLP-2^D/fold-0',
 'dataset-PROTEINS/embedder-MLP-D/fold-0',
 'dataset-PROTEINS/embedder-none/fold-0',
 'dataset-PROTEINS/embedder-QFE-exp/fold-1',
 'dataset-PROTEINS/embedder-QFE-probs/fold-1',
 'dataset-PROTEINS/embedder-MLP-2^D/fold-1',
 'dataset-PROTEINS/embedder-MLP-D/fold-1',
 'dataset-PROTEINS/embedder-none/fold-1',
 'dataset-PROTEINS/embedder-QFE-exp/fold-2',
 'dataset-PROTEINS/embedder-QFE-probs/fold-2',
 'dataset-PROTEINS/embedder-MLP-2^D/fold-2',
 'dataset-PROTEINS/embedder-MLP-D/fold-2',
 'dataset-PROTEINS/embedder-none/fold-2',
 'dataset-PROTEINS/embedder-QFE-exp/fold-3',
 'dataset-PROTEINS/embedder-QFE-probs/fold-3',
 'dataset-PROTEINS/embedder-MLP-2^D/fold-3',
 'dataset-PROTEINS/embedder-MLP-D/fold-3',
 'dataset-PROTEINS/embedder-none/fold-3',
 'dataset-PROTEINS/embedder-QFE-exp/fold-4',
 'dataset-PROTEINS/embedder-QFE-probs/fold-4',
 'dataset-PROTEINS/e

In [3]:
study = optuna.load_study(study_name="dataset-PROTEINS/embedder-none/fold-4", storage=os.getenv('OPTUNA_DB'))
study.best_trials

[FrozenTrial(number=14, state=TrialState.COMPLETE, values=[0.6733977779172682, 0.021348314606741525, 730.0], datetime_start=datetime.datetime(2024, 5, 23, 11, 58, 58, 294074), datetime_complete=datetime.datetime(2024, 5, 23, 11, 59, 49, 611238), params={'pooling': 'sum', 'model': 'GraphSAGE', 'layers': 2, 'hidden': 56, 'dropout': 0.3580313058948699, 'lr': 0.0016670769747313794, 'weight_decay': 3.395416199763484e-05}, user_attrs={}, system_attrs={'nsga2:generation': 0}, intermediate_values={}, distributions={'pooling': CategoricalDistribution(choices=('sum', 'mean', 'max')), 'model': CategoricalDistribution(choices=('GCN', 'GraphConv', 'GraphSAGE', 'GAT')), 'layers': IntDistribution(high=16, log=False, low=1, step=1), 'hidden': IntDistribution(high=256, log=False, low=16, step=1), 'dropout': FloatDistribution(high=0.5, log=False, low=0.0, step=None), 'lr': FloatDistribution(high=0.01, log=True, low=1e-05, step=None), 'weight_decay': FloatDistribution(high=0.001, log=True, low=1e-05, ste

In [4]:
from optuna.visualization import plot_pareto_front

In [5]:
plot_pareto_front(study)

In [6]:
import numpy as np

In [7]:
accuracies = np.array([trial.values[0] for trial in study.best_trials])
errors = np.array([trial.values[1] for trial in study.best_trials])
f1s = np.array([trial.values[2] for trial in study.best_trials])

accuracies_mean = accuracies.mean()
accuracies_std = accuracies.std()
errors_mean = errors.mean()
errors_std = errors.std()
f1s_mean = f1s.mean()
fls_std = f1s.std()

def utility(trial):
    z_acc = (trial.values[0] - accuracies_mean) / accuracies_std
    z_err = -(trial.values[1] - errors_mean) / errors_std
    z_f1 = -(trial.values[2] - f1s_mean) / fls_std

    return z_acc + 3*z_f1 #+ z_err + z_f1

best_trial = max(study.best_trials, key=utility)
best_trial.values

[0.7318121900696755, 0.033356349256167206, 14575.0]

In [8]:
best_trial.number

118

In [9]:
best_trial.params

{'pooling': 'max',
 'model': 'GCN',
 'layers': 6,
 'hidden': 59,
 'dropout': 0.2206078770251621,
 'lr': 0.001734613470581729,
 'weight_decay': 1.5905348635357757e-05}